In [ ]:
cd ../..

In [ ]:
! del /q models\logs\*
! for /d %x in (models\logs\*) do @rd /get_model

In [ ]:
%load_ext tensorboard

In [ ]:
import glob
from datetime import datetime
from os import path

import chess
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
from tensorflow.keras import callbacks, models, utils

from models import get_model
from scripts import moves

In [ ]:
XX, yy_1, yy_2 = [], [], []
for file in glob.glob("data/npz/test/*.npz"):
# for file in glob.glob("data/npz/*/*.npz"):
    with np.load(file) as f:
        XX.append(f["X"]), yy_1.append(f["y_1"]), yy_2.append(f["y_2"])
X, y_1, y_2 = np.concatenate(XX), np.concatenate(yy_1), np.concatenate(yy_2)

In [ ]:
plt.title("Policy Head")
plt.xlabel("Index")
plt.ylabel("Count")
_ = plt.hist(y_1, bins=len(moves))

In [ ]:
plt.title("Value Head")
plt.xlabel("Score")
plt.ylabel("Density")
_, x, _ = plt.hist(y_2, bins=100, density=True)  
_ = plt.plot(x,  stats.gaussian_kde(y_2)(x))

In [ ]:
model = get_model()
model.summary()

In [ ]:
utils.plot_model(
    model,
    to_file=f"models/png/model.png",
    show_shapes=True,
    show_layer_names=False
)

In [ ]:
callback = (
    callbacks.TensorBoard(
        log_dir=f"models/logs/{datetime.now().strftime('%Y%m%d-%H%M%S')}"
    ),
    callbacks.ModelCheckpoint(
        f"models/ser/{datetime.now().strftime('%Y%m%d-%H%M%S')}"
    )
)

In [ ]:
model.compile(
    optimizer="adam",
    loss=("sparse_categorical_crossentropy", "mse")
)

In [ ]:
_ = model.fit(
    x=X, y=(y_1, y_2),
    batch_size=256,
    epochs=5,
    # callbacks=callback,
    validation_split=.2
)

In [ ]:
model.save("models/ser/v4")

In [ ]:
%tensorboard --logdir models/logs